# **Binary Classification of Insurance Cross Selling**

## Downloading Dataset

In [12]:
# @title Kaggle Competition Data Downloading  { display-mode: "form",run :"auto" }
# @markdown Put the competition name
kaggle_username = 'Kaggle_Username'     # @param ['Kaggle_Username', 'kaggle_username', 'kaggle_2']
kaggle_token = "Kaggle"     # @param ["Kaggle", 'kaggle', 'kaggle_2_pass']

project_name = 'playground-series-s4e7'  # @param {type: "string"}


from google.colab import userdata
from IPython import display
import os
token = {"username":userdata.get(kaggle_username),"key":userdata.get(kaggle_token)}
import os, json
os.environ['KAGGLE_CONFIG_DIR']='.'
with open('kaggle.json', "w") as f:
    json.dump(token, f)
!chmod 600 ./kaggle.json
!kaggle competitions download -c $project_name
project_name += ".zip"
!unzip $project_name && rm $project_name
os.environ['MLFLOW_TRACKING_PASSWORD'] = userdata.get('MLFLOW_TRACKING_PASSWORD')
os.environ['MLFLOW_TRACKING_USERNAME'] = userdata.get('MLFLOW_TRACKING_USERNAME')
display.clear_output()
print("Files have been downloaded!")

Files have been downloaded!


In [2]:
!kaggle datasets download annantkumarsingh/health-insurance-cross-sell-prediction-data && unzip *.zip && rm -d *.zip

Dataset URL: https://www.kaggle.com/datasets/annantkumarsingh/health-insurance-cross-sell-prediction-data
License(s): CC0-1.0
 48% 3.00M/6.20M [00:00<00:00, 5.35MB/s]
100% 6.20M/6.20M [00:00<00:00, 8.25MB/s]
Archive:  health-insurance-cross-sell-prediction-data.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: r
new name: test_2.csv
  inflating: test_2.csv              
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: r
new name: original.csv
  inflating: original.csv            


In [3]:
!pip install optuna-dashboard optuna
!pip install mlflow dagshub
!pip install catboost
!mkdir -p output/models
!pip install --upgrade gdown
display.clear_output()
import dagshub
import mlflow
import optuna

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import anderson,kstest, shapiro
from IPython.display import Audio,display as dis
from IPython.display import clear_output

from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge

In [6]:
# plotly libraries setup
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly.io as pio
pio.renderers.default = "colab"
pd.set_option('display.max_columns', None)
# import plotly.offline as pyo
# pyo.init_notebook_mode()

In [7]:
dagshub.init("Kaggle-Competitions-Lab", "SantanuK", mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=5d7a6778-0d45-4450-bcd4-e389ef8e4741&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=36452c4e33df8fa89d9e966839748a6c9a4b3f494540a9330d2395139a122436




Accessing as SantanuK

Initialized MLflow to track repo "SantanuK/Kaggle-Competitions-Lab"

Repository SantanuK/Kaggle-Competitions-Lab initialized!

In [8]:
experiment_name = 'Binary Classification of Insurance Cross Selling'
try:
    mlflow.create_experiment(experiment_name)
    mlflow.set_experiment(experiment_name)
except Exception as e:
    mlflow.set_experiment(experiment_name)
    # print("Experiment has been created or some issue occured!\n","Error: ", e )

In [9]:
from sklearn.metrics import (confusion_matrix, accuracy_score,
                             precision_score, f1_score, recall_score,
                             roc_auc_score, roc_curve, auc
                             )
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import VALID_METRICS

from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import (AdaBoostClassifier, ExtraTreesClassifier,
                              BaggingClassifier, RandomForestClassifier,
                              GradientBoostingClassifier,
                              HistGradientBoostingClassifier,
                              StackingClassifier
                              )
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier, XGBRFClassifier
from catboost import CatBoostClassifier

## EDA

In [19]:
df_train = pd.read_csv('train.csv').drop('id',axis=1)
df_test = pd.read_csv('test.csv').drop('id',axis=1)
df_extra = pd.read_csv('original.csv').drop('id',axis=1).drop_duplicates().reset_index(drop=True)

In [20]:
df_train.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0


In [21]:
df_extra.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


In [25]:
len(df_train), len(df_test), len(df_extra)

(11504798, 7669866, 381109)

In [29]:
result = f""" The number of duplicates presaent in datset:
Number of duplicates in train: {df_train.duplicated().sum()}
Number of duplicates in test : {df_test.duplicated().sum()}
Number of duplicates in extra: {df_extra.duplicated().sum()}
"""
print(result)


Number of duplicates in train: 0
Number of duplicates in test:  0
Number of duplicates in extra: 269



In [22]:
features = df_test.columns
def visualization(dataframe, features=features):
    summary = {
        'Columns'   :   [],
        'Count'     :   [],
        'Unique count': [],
        'Max'       :   [],
        'Min'       :   [],
        'Mean'      :   [],
        'Std'       :   [],
        '5%'        :   [],
        '25%'        :   [],
        '50%'        :   [],
        '75%'        :   [],
        '95%'        :   [],
        # IQR, Low Bound, High Bound
        'IQR'       :   [],
        'Low Bound' :   [],
        'High Bound':   [],
        '< Low Bound' :   [],
        '> High Bound':   [],
        'Outliers'  : [],
        '% Outliers'  : [],
    }
    for col in features:
        summary['Columns'].append(col)
        summary['Count'].append(dataframe[col].notnull().sum())
        summary['Unique count'].append(dataframe[col].nunique())
        summary['Max'].append(dataframe[col].max())
        summary['Min'].append(dataframe[col].min())
        summary['Mean'].append(dataframe[col].mean())
        summary['Std'].append(dataframe[col].std())
        for num in [5, 25, 50, 75, 95]:
            summary[f"{num}%"].append(np.percentile(dataframe[col], num))
        summary['IQR'].append(summary['75%'][-1] - summary['25%'][-1])
        summary['Low Bound'].append(summary['50%'][-1]-1.5*summary['IQR'][-1])
        summary['High Bound'].append(summary['50%'][-1]+1.5*summary['IQR'][-1])

        summary["< Low Bound"].append(len(dataframe[dataframe[col]<summary["Low Bound"][-1]]))
        summary["> High Bound"].append(len(dataframe[dataframe[col]>summary["High Bound"][-1]]))
        summary["Outliers"].append(summary["< Low Bound"][-1]+ summary["> High Bound"][-1])
        summary["% Outliers"].append(summary["Outliers"][-1]/len(dataframe)*100.0)
    summary = pd.DataFrame(summary)
    return summary



In [ ]:
df_train.drop('id', axis=1)

In [13]:
num_features =[]
cat_features =[]
target_feature = 'FloodProbability'

In [ ]:
def select_model(dataframe, test_size=.3, target_feature=target_feature, model = None):
    !mkdir -p output/data
    algos = [
            #  SVC(), NuSVC(),
            #  LinearRegression(),
            DecisionTreeClassifier(), ExtraTreeClassifier(),
            AdaBoostClassifier(), BaggingClassifier(),
            ExtraTreesClassifier(), RandomForestClassifier(),
            GradientBoostingClassifier(),
            HistGradientBoostingClassifier(),
            LGBMClassifier(), XGBClassifier(), XGBRFClassifier(),
            CatBoostClassifier()
            ]
    names = list(map(lambda x: x.__class__.__name__, algos))

    history = { 'algo_name': names,
                'train_score' : [],
                'test_score' : [],
                'train_metrics': [],
                'test_metrics': [],
                }
    X_train, X_test, y_train, y_test = train_test_split(dataframe.drop(target_feature, axis=1), dataframe[target_feature], test_size=test_size)
    for model in algos:
        print(f"{model.__class__.__name__} has started!")
        model.fit(X_train, y_train)
        pred_train = model.predict(X_train)
        pred_test = model.predict(X_test)
        history['train_score'].append(accuracy_score(y_train, pred_train))
        history['test_score'].append(accuracy_score(y_test, pred_test))
        try:
            y_prob = model.predict_proba(X_train)[:, 1]
            roc_auc = roc_auc_score(y_train, y_prob)
            history['train_metrics'].append(roc_auc)
            y_prob_ = model.predict_proba(X_test)[:, 1]
            roc_auc_ = roc_auc_score(y_test, y_prob_)
            history['test_metrics'].append(roc_auc_)
        except Exception as e:
            history['train_metrics'].append(np.nan)
            history['test_metrics'].append(np.nan)
    return pd.DataFrame(history)